<a href="https://colab.research.google.com/github/Dylph/eda-on-google-merchandise-store/blob/main/EDA_Bigquery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
import os
from google.cloud import bigquery

import numpy as np
import pandas as pd
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import plotly.io as pio
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Set the environment variable
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/path/to/your/keyfile.json"

# Set up the BigQuery client
client = bigquery.Client()

# This function runs a query and returns the results as a DataFrame
def run_query(query):
    query_job = client.query(query)
    return query_job.to_dataframe()

## Understanding Dataset Contents and Structure

In [62]:
# Get the date range
query = f"""
SELECT 
  MIN(date) as start_date, MAX(date) as end_date
FROM 
  `bigquery-public-data.google_analytics_sample.ga_sessions_*`
"""

date_range = run_query(query)
date_range

,start_date,end_date
0,20160801,20170801


In [63]:
# Show one table from the dataset
query = f"""
SELECT 
  *
FROM
  `bigquery-public-data.google_analytics_sample.ga_sessions_20170801`
LIMIT
  10
"""

sample_table = run_query(query)
sample_table.head()

,visitorId,visitNumber,visitId,visitStartTime,date,totals,trafficSource,device,geoNetwork,customDimensions,hits,fullVisitorId,userId,clientId,channelGrouping,socialEngagementType
0,<NA>,1,1501591568,1501591568,20170801,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Europe', 'subContinent': 'South...",[],"[{'hitNumber': 1, 'time': 0, 'hour': 5, 'minut...",3418334011779872055,None,None,Organic Search,Not Socially Engaged
1,<NA>,2,1501589647,1501589647,20170801,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","{'referralPath': '/analytics/web/', 'campaign'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Asia', 'subContinent': 'Souther...","[{'index': 4, 'value': 'APAC'}]","[{'hitNumber': 1, 'time': 0, 'hour': 5, 'minut...",2474397855041322408,None,None,Referral,Not Socially Engaged
2,<NA>,1,1501616621,1501616621,20170801,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","{'referralPath': '/analytics/web/', 'campaign'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Europe', 'subContinent': 'North...","[{'index': 4, 'value': 'EMEA'}]","[{'hitNumber': 1, 'time': 0, 'hour': 12, 'minu...",5870462820713110108,None,None,Referral,Not Socially Engaged
3,<NA>,1,1501601200,1501601200,20170801,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","{'referralPath': '/analytics/web/', 'campaign'...","{'browser': 'Firefox', 'browserVersion': 'not ...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 8, 'minut...",9397809171349480379,None,None,Referral,Not Socially Engaged
4,<NA>,1,1501615525,1501615525,20170801,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","{'referralPath': '/analytics/web/', 'campaign'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 12, 'minu...",6089902943184578335,None,None,Referral,Not Socially Engaged


## 1. What channels drive the most visitors?

In [64]:
# What channels drive the most visitors?
query = f"""
SELECT channelGrouping, COUNT(*) as visitor_count
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
GROUP BY channelGrouping
ORDER BY visitor_count DESC
"""

channel_visitors = run_query(query)
print(channel_visitors)

  channelGrouping  visitor_count
0  Organic Search         381561
1          Social         226117
2          Direct         143026
3        Referral         104838
4     Paid Search          25326
5      Affiliates          16403
6         Display           6262
7         (Other)            120


In [65]:
# Sort the dataframe by visitor_count in descending order
channel_visitors_sorted = channel_visitors.sort_values(by='visitor_count', ascending=False)

# Define a list of custom colors
custom_color_scale = ['#CAF0F8', '#ADE8F4', '#90E0EF', '#48CAE4', '#00B4D8', '#0096C7', '#0077B6', '#023E8A', '#03045E']

# Create a horizontal bar chart using Plotly and order by visitor counts
fig = px.bar(channel_visitors_sorted,
             x='visitor_count',
             y='channelGrouping',
             title='What channels drive the most visitors?', 
             orientation='h',
             category_orders={'channelGrouping': channel_visitors_sorted['channelGrouping'].tolist()},
             color='visitor_count',
             color_continuous_scale=custom_color_scale)

# Set the color of the grid lines to gray
fig.update_xaxes(gridcolor='#4a4e69', zerolinecolor='#4a4e69')

# Set the Title of the axes
fig.update_xaxes(title='Visitor Count')  # This sets the X axis title
fig.update_yaxes(title='Channel Group',   # This sets the Y axis title
                 ticksuffix = "  ")

# Customize the plot background color
fig.update_layout(
    plot_bgcolor='#212529',  # Set the custom plot background color
    paper_bgcolor='#212529',  # Set the custom plot paper background color
    font_color='white',
        
    # Adjust the spacing between bars and labels
    barmode='group',
    bargap=0.4,
)

# Remove the border color for the bars
fig.update_traces(marker=dict(line_color='rgba(0,0,0,0)'))

# Update the color scale title
fig.update_coloraxes(colorbar_title='Visitor Count')  # Replace 'Custom Title' with your desired title

# Show the chart
fig.show()

## 2. Of the people visiting my website, what percentage of them are new visitors compared to returning visitors?

In [66]:
# Of the people visiting my website, what percentage of them are new visitors compared to returning visitors?
query = """
SELECT
  IF(totals.newVisits = 1, 'New Visitor', 'Returning Visitor') AS visitor_type,
  COUNT(*) AS visitors
FROM
  `bigquery-public-data.google_analytics_sample.ga_sessions_*`
GROUP BY
  visitor_type
"""

new_v_return = run_query(query)
print(new_v_return)

        visitor_type  visitors
0  Returning Visitor    200593
1        New Visitor    703060


In [67]:
# Get the data
new_visitors = new_v_return['visitors'][0]
returning_visitors = new_v_return['visitors'][1]

# Create the pie chart
fig = px.pie(new_v_return, values='visitors', names='visitor_type', color='visitor_type')

# Set custom colors
colors = ['royalblue', 'lightcyan']
fig.update_traces(marker=dict(colors=colors))

# Resize the pie chart to a square form
fig.update_layout(width=600, height=400)

# Set the title
fig.update_layout(title='Visitor Type')

# Customize the plot background color
fig.update_layout(
    plot_bgcolor='#212529',  # Set the custom plot background color
    paper_bgcolor='#212529',  # Set the custom plot paper background color
    font_color='white',
)

# Show the values along with the percentages
fig.update_traces(textinfo='value+percent', textfont_size=14)

# Show the pie chart
fig.show()

## 3. How many times do visitors complete the goal?

In [68]:
# How many times do visitors complete the goal?
query = """
SELECT
  PARSE_DATE('%Y%m%d', _TABLE_SUFFIX) AS date,
  COUNT(DISTINCT CASE
      WHEN REGEXP_CONTAINS(hits.page.pagePath, '/ordercompleted') THEN CONCAT(CAST(fullVisitorId AS STRING), CAST(visitStartTime AS STRING))
      ELSE NULL
    END) AS destination_goals,
  COUNT(DISTINCT CASE
      WHEN hits.time >= 300000 THEN CONCAT(CAST(fullVisitorId AS STRING), CAST(visitStartTime AS STRING))
      ELSE NULL
    END) AS duration_goals,
  COUNT(DISTINCT CASE
      WHEN totals.pageviews >= 3 THEN CONCAT(CAST(fullVisitorId AS STRING), CAST(visitStartTime AS STRING))
      ELSE NULL
    END) AS pages_per_visit_goals,
  COUNT(DISTINCT CASE
      WHEN REGEXP_CONTAINS(hits.eventInfo.eventAction, '(Product Click|Add to Cart)') THEN CONCAT(CAST(fullVisitorId AS STRING), CAST(visitStartTime AS STRING))
      ELSE NULL
    END) AS event_goals
FROM
  `bigquery-public-data.google_analytics_sample.ga_sessions_*` AS sessions,
  UNNEST(hits) AS hits
WHERE
  _TABLE_SUFFIX BETWEEN '20160801' AND '20170801'
  AND hits.type IN ('PAGE', 'EVENT')
GROUP BY
  date
ORDER BY
  date
"""

# Execute the query and store the results in a DataFrame
goals = run_query(query)

In [69]:
# Define the summary table
summary_goals = goals.describe()

# Drop the 'date' column before aggregation
goals_without_date = goals.drop(columns=['date'])

# Calculate the sum using .agg() and create a DataFrame for it
goals_sum = pd.DataFrame(goals_without_date.agg(['sum']), index=['sum'])

# Concatenate the summary statistics DataFrame with the sum DataFrame
summary_goals_with_sum = pd.concat([summary_goals, goals_sum])

summary_goals_with_sum

,destination_goals,duration_goals,pages_per_visit_goals,event_goals
count,366.0,366.0,366.0,366.0
mean,33.587432,276.251366,839.510929,224.409836
std,16.279974,71.813931,223.967741,68.834137
min,4.0,104.0,417.0,84.0
25%,21.0,216.0,668.0,173.0
50%,33.0,285.0,858.0,221.0
75%,42.0,324.0,972.75,272.75
max,92.0,470.0,1684.0,415.0
sum,12293.0,101108.0,307261.0,82134.0


In [70]:
# Custom color scale
custom_color_scale = ['#00FFFF', '#00BFFF','#4682B4', '#1E90FF']

# Plotting the line chart
fig = px.line(goals, x='date', y=['destination_goals', 'duration_goals', 'pages_per_visit_goals', 'event_goals'],
              labels={'value': 'Number of Goals Achieved', 'variable': 'Goals', 'date': 'Date'},
              title='Goal Achievements Over Time',
              color_discrete_sequence=custom_color_scale)

# Set the color of the grid lines to gray
fig.update_xaxes(gridcolor='#4a4e69')
fig.update_yaxes(gridcolor='#4a4e69', zerolinecolor='#4a4e69')

# Customize the plot background color
fig.update_layout(
    plot_bgcolor='#212529',  # Set the custom plot background color
    paper_bgcolor='#212529',  # Set the custom plot paper background color
    font_color='white',  # Set the custom font color
)

# Display the chart
fig.show()

## What's the overall goal conversion rate?

In [71]:
# What's the overall goal conversion rate?
query = """
SELECT
  PARSE_DATE('%Y%m%d', _TABLE_SUFFIX) AS date,
  COUNT(DISTINCT CONCAT(CAST(fullvisitorid as string), CAST(visitstarttime as string))) AS total_sessions
FROM
  `bigquery-public-data.google_analytics_sample.ga_sessions_*` as sessions,
  UNNEST(hits) as hits
WHERE
  _TABLE_SUFFIX BETWEEN '20160801' AND '20170801'
  AND hits.type IN ('PAGE', 'EVENT')
GROUP BY
  date
ORDER BY
  date
"""

# Execute the query and store the results in a DataFrame
total_sessions = run_query(query)

In [72]:
# Merge the total_sessions and goals DataFrames on the date column
conversion_rates_data = goals.merge(total_sessions, on='date', suffixes=('', '_total'))

# Calculate the conversion rates for each goal
conversion_rates_data['destination_conversion_rate'] = conversion_rates_data['destination_goals'] / conversion_rates_data['total_sessions']
conversion_rates_data['duration_conversion_rate'] = conversion_rates_data['duration_goals'] / conversion_rates_data['total_sessions']
conversion_rates_data['pages_per_visit_conversion_rate'] = conversion_rates_data['pages_per_visit_goals'] / conversion_rates_data['total_sessions']
conversion_rates_data['event_conversion_rate'] = conversion_rates_data['event_goals'] / conversion_rates_data['total_sessions']

# Calculate the average conversion rate for each goal
average_destination_goal_conversion_rate = conversion_rates_data['destination_conversion_rate'].mean()*100
average_duration_goal_conversion_rate = conversion_rates_data['duration_conversion_rate'].mean()*100
average_pages_per_visit_goal_conversion_rate = conversion_rates_data['pages_per_visit_conversion_rate'].mean()*100
average_event_goal_conversion_rate = conversion_rates_data['event_conversion_rate'].mean()*100

# convert the results into a Dataframe
data = {
    'Goal': ['destination_goals', 'duration_goals', 'pages_per_visit_goals', 'event_goals'],
    'Average Conversion Rate (%)': [average_destination_goal_conversion_rate, average_duration_goal_conversion_rate, average_pages_per_visit_goal_conversion_rate, average_event_goal_conversion_rate]
}

average_conversion_rate = pd.DataFrame(data)

# calculate the mean of the goals metics
average_goals = goals.mean(axis=0, numeric_only=True)
average_goals = average_goals.to_frame().reset_index()
average_goals.columns = ['Goal', 'Average Daily Goals']

In [73]:
# Join the dataframes on the 'Goal' column
combined_df = average_goals.merge(average_conversion_rate, on='Goal')

# Display the combined dataframe
combined_df

,Goal,Average Daily Goals,Average Conversion Rate (%)
0,destination_goals,33.587432,1.357967
1,duration_goals,276.251366,11.443141
2,pages_per_visit_goals,839.510929,34.476752
3,event_goals,224.409836,9.273924


## 5. How long do visitors stay on my website? How many pages do visitors view?

In [74]:
# How long do visitors stay on my website? How many pages do visitors view?
query = """
SELECT
  AVG(totals.timeOnSite) AS avg_time_on_site,
  AVG(totals.pageviews) AS avg_pageviews
FROM
  `bigquery-public-data.google_analytics_sample.ga_sessions_*`
"""

# Execute the query and store the results in a DataFrame
visitor_time = run_query(query)
visitor_time

,avg_time_on_site,avg_pageviews
0,262.612141,3.849764


## 6. What is the most often purchased item based on revenue?

In [75]:
# What is the most often purchased item based on revenue?
query = """
SELECT
  product.v2ProductName AS product_name,
  SUM(product.productRevenue) / 1e6  AS total_revenue
FROM
  `bigquery-public-data.google_analytics_sample.ga_sessions_*`,
  UNNEST(hits) AS hits,
  UNNEST(hits.product) AS product
WHERE
  hits.eCommerceAction.action_type = "6"
GROUP BY
  product_name
ORDER BY
  total_revenue DESC
LIMIT 10
"""

# Execute the query and store the results in a DataFrame
item_purchased = run_query(query)
item_purchased

,product_name,total_revenue
0,Google Men's Zip Hoodie,47636.271605
1,26 oz Double Wall Insulated Bottle,44454.491527
2,Google 22 oz Water Bottle,42995.041996
3,Leatherette Journal,38563.457656
4,Google Sunglasses,35490.321293
5,Google Metallic Notebook Set,28846.596814
6,Google Men's 100% Cotton Short Sleeve Hero Tee...,28047.612114
7,Recycled Paper Journal Set,27918.170430
8,Google Men's 100% Cotton Short Sleeve Hero Tee...,27856.018891
9,Google Hard Cover Journal,25432.673189


In [76]:
# Sort the DataFrame by 'total_revenue' in descending order
item_purchased_sorted = item_purchased.sort_values(by='total_revenue', ascending=True)

fig = px.bar(item_purchased_sorted, 
             x='total_revenue', 
             y='product_name', 
             orientation='h',
             labels={'product_name': 'Product Name', 'total_revenue': 'Total Revenue'},
             title='Top 10 Most Often Purchased Items Based on Revenue',
             color='total_revenue',
             color_continuous_scale=custom_color_scale)

# Set the color of the grid lines to gray
fig.update_xaxes(gridcolor='#4a4e69', zerolinecolor='#4a4e69')

# Set the Title of the axes
fig.update_xaxes(title='Visitor Count')  # This sets the X axis title
fig.update_yaxes(title='Channel Group', # This sets the Y axis title
                 ticksuffix = "  ") 

# Customize the plot background color
fig.update_layout(
    plot_bgcolor='#212529',  # Set the custom plot background color
    paper_bgcolor='#212529',  # Set the custom plot paper background color
    font_color='white',
        
    # Adjust the spacing between bars and labels
    barmode='group',
    bargap=0.6,
)

# Remove the border color for the bars
fig.update_traces(marker=dict(line_color='rgba(0,0,0,0)'))

# Display the chart
fig.show()

## 7. What is the Total Transactions and Revenue by Day of the Week??

In [77]:
# What is the Total Transactions and Revenue by Day of the Week?
query = """
SELECT
  EXTRACT(DAYOFWEEK FROM PARSE_DATE('%Y%m%d', date)) AS day_of_week,
  COUNT(*) AS total_transactions,
  SUM(totals.totalTransactionRevenue) / 1e6 AS total_revenue
FROM
  `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
  totals.totalTransactionRevenue IS NOT NULL
GROUP BY
  day_of_week
ORDER BY
  day_of_week
"""

transactions_by_day = run_query(query)
transactions_by_day

,day_of_week,total_transactions,total_revenue
0,1,947,85222.55
1,2,2054,300155.24
2,3,2020,373523.25
3,4,1931,349050.07
4,5,1902,316539.19
5,6,1879,286258.96
6,7,782,69399.97


In [78]:
fig = px.bar(transactions_by_day, 
             x='day_of_week', 
             y='total_transactions', 
             text='total_transactions')

# Customize bar color
fig.update_traces(marker=dict(color='#3a86ff'))

# Add the line chart for revenue with secondary y-axis
fig.add_scatter(x=transactions_by_day['day_of_week'], 
                y=transactions_by_day['total_revenue'], 
                name='Revenue',
                yaxis='y2',
                line=dict(color='#90E0EF', width=4),
                marker=dict(color='#FCF6BD'))

fig.update_layout(
    title='Total Transactions and Revenue by Day of Week',
    xaxis=dict(title='Day of Week'),
    yaxis=dict(title='Total Transactions', gridcolor='rgba(0,0,0,0)', zerolinecolor='#4A4E69'),
    yaxis2=dict(title='Total Revenue', overlaying='y', side='right', gridcolor='#4A4E69'),

    # Customize the plot background color
    plot_bgcolor='#212529',  # Set the custom plot background color
    paper_bgcolor='#212529',  # Set the custom plot paper background color
    font_color='white',
        
    # Adjust the spacing between bars and labels
    barmode='group',
    bargap=0.7,

    # Update the legend position
    legend=dict(x=0.85, y=1)
)

# Remove the border color for the bars
fig.update_traces(marker=dict(line_color='rgba(0,0,0,0)'))

# Add a separate bar trace for the legend
legend_bar_trace = fig.add_bar(x=[None], y=[None], marker=dict(color='#4361EE', line_color='rgba(0,0,0,0)'), showlegend=True, name='Total Transactions')

fig.show()

## 8. What is the weekly revenue?

In [79]:
# What is the weekly revenue?
query = """
SELECT
  EXTRACT(YEAR FROM PARSE_DATE('%Y%m%d', date)) AS year,
  EXTRACT(WEEK FROM PARSE_DATE('%Y%m%d', date)) AS week,
  SUM(totals.totalTransactionRevenue) / 1e6 AS weekly_revenue
FROM
  `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
  totals.totalTransactionRevenue IS NOT NULL
GROUP BY
  year, week
ORDER BY
  year, week
"""

weekly_revenue = run_query(query)

In [80]:
# Create a combined 'year_week' column
weekly_revenue['year_week'] = pd.to_datetime(weekly_revenue['year'].astype(str) + weekly_revenue['week'].astype(str) + '-1', format='%Y%W-%w')

# Sort the DataFrame by the 'year_week' column
weekly_revenue = weekly_revenue.sort_values('year_week')

# Reformat 'year_week' column
weekly_revenue['year_week'] = weekly_revenue['year_week'].dt.strftime('%Y-%W')

# Define custom color palette
custom_colors = ['#E63946', '#0096C7']

# Plot the line chart using Plotly Express
fig = px.line(weekly_revenue, x='year_week', y='weekly_revenue', 
              labels={'weekly_revenue': 'Revenue', 'year_week': 'Year in Weeks'},
              title='Weekly Revenue',
              color='year',
              color_discrete_sequence=custom_colors)

# Create custom legend lables
legend_name = {'2016':'Aug - Dec 2016', '2017': 'Jan - Jul 2017'}
fig.for_each_trace(lambda t: t.update(name = legend_name[t.name],
                                      legendgroup = legend_name[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, legend_name[t.name])
                                     )
                  )

# Set the color of the grid lines to gray
fig.update_xaxes(type='category', gridcolor='rgba(0,0,0,0)')  # Set x-axis type to categorical     
fig.update_yaxes(gridcolor='#4A4E69')

# Customize the plot background color
fig.update_layout(
    plot_bgcolor='#212529',  # Set the custom plot background color
    paper_bgcolor='#212529',  # Set the custom plot paper background color
    font_color='white',  # Set the custom font color

    # Update the legend position
    legend=dict(x=0.9, y=1)
)

# Display the chart
fig.show()

In [81]:
# What is the monthly revenue?
query = """
SELECT
  EXTRACT(YEAR FROM PARSE_DATE('%Y%m%d', date)) AS year,
  EXTRACT(MONTH FROM PARSE_DATE('%Y%m%d', date)) AS month,
  SUM(totals.totalTransactionRevenue) / 1e6 AS monthly_revenue
FROM
  `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
   _TABLE_SUFFIX BETWEEN '20160801' AND '20170731'
GROUP BY
  year, month
ORDER BY
  year, month
"""

monthly_revenue = run_query(query)

In [82]:
# Create a combined 'year_monthk' column
monthly_revenue['year_month'] = pd.to_datetime(monthly_revenue['year'].astype(str) + monthly_revenue['month'].astype(str) + '-1', format='%Y%m-%d')

# Sort the DataFrame by the 'year_month' column
monthly_revenue = monthly_revenue.sort_values('year_month')

# Reformat 'year_month' column
monthly_revenue['year_month'] = monthly_revenue['year_month'].dt.strftime('%Y-%m')

# Create the bar chart using Plotly Express
fig = px.bar(monthly_revenue, 
             x='year_month', 
             y='monthly_revenue', 
             barmode='group')

# Set the color of the grid lines to gray
fig.update_yaxes(gridcolor='#4a4e69', zerolinecolor='#4a4e69')

# Customize the chart layout
fig.update_layout(
    title='Monthly Revenue',
    xaxis_title='Month',
    yaxis_title='Revenue',
    legend_title='Year',

    # Customize the plot background color
    plot_bgcolor='#212529',  # Set the custom plot background color
    paper_bgcolor='#212529',  # Set the custom plot paper background color
    font_color='white',
        
    # Adjust the spacing between bars and labels
    barmode='group',
    bargap=0.7,
)

# Set custom colors for the bars
n = len(monthly_revenue)
colors = ['#E63946'] * 5 + ['#0096C7'] * (n - 5)

# Update the bar trace with custom colors and Remove the border color for the bars
fig.update_traces(marker=dict(color=colors, 
                              line_color='rgba(0,0,0,0)'))

# Display the chart
fig.show()

## 9. Which country is the biggest customer?

In [83]:
# Which country is the biggest customer?
query="""
SELECT 
    geoNetwork.country AS country,
    COUNT (totals.visits) AS visitors,
    COUNT(CASE WHEN hits.eCommerceAction.action_type = '6' THEN fullvisitorId ELSE NULL END ) AS customers
FROM 
    `bigquery-public-data.google_analytics_sample.ga_sessions_*`,
    UNNEST (hits) AS hits
GROUP BY 
    country 
ORDER BY 
    customers DESC 
"""

country_data = run_query(query)
country_data

,country,visitors,customers
0,United States,2483593,23159
1,Canada,159092,404
2,Venezuela,21056,316
3,United Kingdom,107145,40
4,Mexico,38057,40
...,...,...,...
217,Maldives,203,0
218,Malta,571,0
219,Grenada,91,0
220,Dominica,11,0


In [84]:
# Filter the top 10 countries
country_data = country_data.sort_values(by='visitors', ascending=False)
top_10_countries = country_data.head(10)

# Aggregate the remaining countries' data into an 'Others' category
others = pd.DataFrame([{
    'country': 'Others',
    'visitors': country_data[10:]['visitors'].sum(),
    'customers': country_data[10:]['customers'].sum()
}])

# Combine the top 10 countries with the 'Others' category
final_data = pd.concat([top_10_countries, others], ignore_index=True)

# Define a list of custom colors
custom_color_scale = ['#1976D2', '#42A5F5', '#64B5F6', '#90CAF9', '#BBDEFB', '#E3F2FD', '#F6CACC', '#F1A7A9', '#EC8385', '#E66063', '#E35053']

# Create the donut chart for visitors
visitors_pie = go.Pie(labels=final_data['country'], 
                      values=final_data['visitors'], 
                      name='Visitors', 
                      hole=0.75,
                      marker=dict(colors=custom_color_scale)
                      )

# Calculate the percentage for each customer category
total_customers = final_data['customers'].sum()
final_data['percentage'] = (final_data['customers'] / total_customers) * 100

# Create hover text with percentage information
hover_text = [f"{country}<br>{percentage:.2f}%" for country, percentage in zip(final_data['country'], final_data['percentage'])]

# Create the treemap chart for customers
customers_treemap = go.Treemap(
    labels=final_data['country'],
    parents=[''] * len(final_data),
    values=final_data['customers'],
    name='Customers',
    textinfo='label',
    hovertext=hover_text,
    hoverinfo='text+value',
    marker=dict(colors=custom_color_scale)
)


# Combine both charts using make_subplots
fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'pie'}, {'type': 'treemap'}]], subplot_titles=['Visitors', 'Customers'])
fig.add_trace(visitors_pie, row=1, col=1)
fig.add_trace(customers_treemap, row=1, col=2)

# Update the layout and show the plot
fig.update_layout(
    title='Top Countries with Most Visitors and Customers',
    
    # Customize the plot background color
    plot_bgcolor='#212529',  # Set the custom plot background color
    paper_bgcolor='#212529',  # Set the custom plot paper background color
    font_color='white',

    # Update the legend position
    legend=dict(
        x=0.46,  # Set the x position of the legend
        y=0.5,  # Set the y position of the legend
        xanchor='center',  # Anchor the legend at the center of the x position
        yanchor='middle',  # Anchor the legend at the middle of the y position
        orientation='v',  # Set the orientation of the legend to vertical
        bgcolor='#778DA9'  # Set the background color of the legend with some transparency
    )
)

fig.show()